Функция на вход получает список белков, и выдает граф коэкспресси этих белков. Дефолтный граф - полный.

## Примеры работы функции

Аргументы:
- proteins список белков
- table используемая таблица
- support минимальная поддержка данных (насколько коэпрессия подтверждена, от 0 до 3, по умолчанию 3)
- N минимальный уровень корреляции (от 0 до бесконечности, чем меньше - тем более скоррелировано, по умолчанию 2000)
- view (используемый модуль питона, "b" - bokeh или "c" - ipycytoscape)

In [6]:
protein_tree()

Loading BokehJS ...

In [7]:
alles

,Source protein,Target protein,Source gene,Target gene,Supportability,Mean MR,Function
0,von Willebrand factor,Cadherin-5,VWF,CDH5,3,210.3375,cadherin 5
1,Vascular endothelial growth factor D,Cadherin-5,VEGFD,CDH5,3,7098.9675,cadherin 5
2,Vascular cell adhesion protein 1,Cadherin-5,VCAM1,CDH5,3,7609.2925,cadherin 5
3,Vasorin,Cadherin-5,VASN,CDH5,3,7451.3450,cadherin 5
4,Tumor necrosis factor receptor superfamily mem...,Cadherin-5,TNFRSF1B,CDH5,3,5594.2600,cadherin 5
...,...,...,...,...,...,...,...
17598,Fibrinogen gamma chain,Mucin-16,FGG,MUC16,3,9114.1650,mucin 16: cell surface associated
17599,Desmoplakin,Mucin-16,DSP,MUC16,3,7669.6675,mucin 16: cell surface associated
17600,Ceruloplasmin,Mucin-16,CP,MUC16,3,4529.2525,mucin 16: cell surface associated
17601,Complement C3,Mucin-16,C3,MUC16,3,6014.1700,mucin 16: cell surface associated


In [13]:
protein_tree(proteins = ['Fibrinogen gamma chain', 'Complement C3'])

Loading BokehJS ...

In [14]:
protein_tree(proteins = ['Fibrinogen gamma chain', 'Complement C3'], N = 5000)

Loading BokehJS ...

In [16]:
protein_tree(proteins = ['Fibrinogen gamma chain', 'Complement C3'], view = "c")

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'font-famil…

In [1]:
# загрузка необходимых пакетов

import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.plotting import output_file
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

import ipycytoscape
import ipywidgets
import json

In [2]:
alles = pd.read_csv("coexpression_our_only.tsv", sep = "\t")
prs = list(set(alles["Source protein"]))

In [3]:
def protein_tree(proteins = prs, table = alles, support = 3, N = 2000, view = "b"):
    table = table[table["Supportability"] >= support]
    table = table[table["Mean MR"] <= N]
    proteins = pd.DataFrame(proteins)
    proteins.columns = ["Source protein"]
    table = table.merge(proteins, on = "Source protein")
    
    G = networkx.from_pandas_edgelist(table, "Source protein", "Target protein")
    if view == "b":
        # аутпут в ноутбук
        output_notebook()

        # подгонка размера и цвета
        degrees = dict(networkx.degree(G))
        networkx.set_node_attributes(G, name='degree', values=degrees)
        number_to_adjust_by = 5
        adjusted_node_size = dict([(node, 0.3*degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
        networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)
        size_by_this_attribute = 'adjusted_node_size'
        color_by_this_attribute = 'adjusted_node_size'

        # Blues8, Reds8, Purples8, Oranges8, Viridis8
        color_palette = Reds8

        title = 'Protein-disease associations'

        # hovering
        HOVER_TOOLTIPS = [("Type", "@index"), ("Degree", "@degree")]

        plot = figure(tooltips = HOVER_TOOLTIPS,
        tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
        x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

        # network graph object
        network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

        # node sizes and colors according to node degree
        minimum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        maximum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
        network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

        # edge opacity and width
        network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

        plot.renderers.append(network_graph)

        show(plot)
    
    elif view == "c":
        
        my_style = [
    {'selector': 'node','style': {
        'font-family': 'arial',
        'font-size': '10px',
        'label': 'data(id)',
        'background-color': 'red'}}]
        
        ipycytoscape_obj = ipycytoscape.CytoscapeWidget()
        ipycytoscape_obj.graph.add_graph_from_networkx(G)
        ipycytoscape_obj.set_style(my_style)
        ipycytoscape_obj.set_style(my_style)
        return ipycytoscape_obj